In [46]:
from Strategies.StrategyBB import StrategyBB
from Strategies.StrategyMACD import StrategyMACD
from Strategies.StrategyOBV import StrategyOBV
from Strategies.StrategyRSI import StrategyRSI
from Strategies.StrategyST import StrategyST
from historyData.HistoryData import HistoryData
from tinkoff.invest.grpc.marketdata_pb2 import CandleInterval
from datetime import datetime

import asyncio

In [47]:
candle_interval = CandleInterval.CANDLE_INTERVAL_5_MIN
data_set = HistoryData()

In [48]:
from time import sleep

candles_down = list()
candles_up = list()
candles_flat = list()
candles_short_up = list()
candles_short_down = list()
candles_7_period = list()
candles_6_period = list()
candles_5_period = list()
candles_4_period = list()
candles_3_period = list()

async def data_get():
    global candles_down, candles_up, candles_flat, candles_short_up, candles_short_down, candles_7_period, candles_6_period, candles_5_period, candles_4_period, candles_3_period
    
    candles_short_up = await (data_set.get_tinkoff_server_data(datetime(2023, 12, 1), datetime(2024, 1, 1), candle_interval))
    candles_short_down = await (data_set.get_tinkoff_server_data(datetime(2023, 12, 2), datetime(2024, 1, 1), candle_interval))
    candles_down = await (data_set.get_tinkoff_server_data(datetime(2023, 11, 2), datetime(2023, 12, 1), candle_interval))
    candles_up = await (data_set.get_tinkoff_server_data(datetime(2023, 8, 2), datetime(2023, 9, 1), candle_interval))
    candles_flat = await (data_set.get_tinkoff_server_data(datetime(2023, 9, 2), datetime(2023, 10, 1), candle_interval))
    candles_7_period = await (data_set.get_tinkoff_server_data(datetime(2023, 7, 2), datetime(2023, 8, 1), candle_interval))
    candles_6_period = await (data_set.get_tinkoff_server_data(datetime(2023, 6, 2), datetime(2023, 7, 1), candle_interval))
    candles_5_period = await (data_set.get_tinkoff_server_data(datetime(2023, 5, 2), datetime(2023, 6, 1), candle_interval))
    candles_4_period = await (data_set.get_tinkoff_server_data(datetime(2023, 4, 2), datetime(2023, 5, 1), candle_interval))
    candles_3_period = await (data_set.get_tinkoff_server_data(datetime(2023, 3, 2), datetime(2023, 4, 1), candle_interval))

# Run the event loop directly using await
await data_get()

# Creates a Data Set For Future Model

In [61]:
all_candles = [candles_down, candles_up, candles_flat, candles_short_up, candles_short_down, candles_7_period, candles_6_period, candles_5_period, candles_4_period, candles_3_period]

for i, candles in enumerate(all_candles):
    saver = list()
    for candle in candles:
        if datetime.strptime(str(candle.time).replace('+00:00', ''), '%Y-%m-%d %H:%M:%S').weekday() in [5, 6]:
            continue
        saver.append(candle)
    all_candles[i] = saver

len(all_candles[0])

3444

In [68]:
import numpy as np
from AppLogic.Settings import Settings
from tinkoff.invest.utils import quotation_to_decimal

commission = 0.0005
all_data_index_buy_sell = []

for candles in all_candles:
    data_index_buy_sell = []
    total_earn = 0
    total_sum_earned = 0
    bought_at = -1
    for i in range(len(list(candles))):
        bought_at = float(quotation_to_decimal(candles[i].close))
        this_candle_earned = 0
        for j in range(i + 1, len(candles)):
            further_candle_price = float(quotation_to_decimal(candles[j].close))
            this_candle_earned = bought_at - further_candle_price

            if this_candle_earned > 1.5 / 100 * bought_at:  # 0.1 is stop loss
                data_index_buy_sell.append({"bought": i})
                break
            elif this_candle_earned < -0.75 / 100 * bought_at:  # 0.05 is stop loss
                break
    all_data_index_buy_sell.append(data_index_buy_sell)
    print(f"Length of the marked data: {len(data_index_buy_sell)}", f"total length: {len(candles)}")
                
# for candles in all_candles:
#     data_index_buy_sell = []
#     total_earn = 0
#     total_sum_earned = 0
#     bought_at = -1
#     counter = 0
#     i = 13
#     up = 0.1
#     down = -0.05
# 
#     while i < len(list(candles)) - 2:
#         i += 1
# 
#         current_candle_price = float(quotation_to_decimal(candles[i].close))
#         next_candle_price = float(quotation_to_decimal(candles[i + 1].close))
# 
#         if current_candle_price < next_candle_price and bought_at == -1:
#             data_index_buy_sell.append({"bought": i})
#             bought_at = current_candle_price
#             counter = i  
#         elif bought_at != -1 and bought_at - current_candle_price < down / 100 * bought_at:
#             data_index_buy_sell.pop(-1)
#             i = counter + 1
#             bought_at = -1    
#         elif bought_at != -1 and bought_at - current_candle_price > up / 100 * bought_at:  # 0.1 is stop loss
#             data_index_buy_sell.append({"sold": i})
#             bought_at = -1

Length of the marked data: 669 total length: 3444
Length of the marked data: 1003 total length: 3608
Length of the marked data: 1218 total length: 3280
Length of the marked data: 882 total length: 3444
Length of the marked data: 785 total length: 3280
Length of the marked data: 814 total length: 3444
Length of the marked data: 1451 total length: 3388
Length of the marked data: 914 total length: 3444
Length of the marked data: 367 total length: 3280
Length of the marked data: 689 total length: 3550


In [69]:
import pandas as pd

all_new_data_buy = list()
for data_index_buy_sell in all_data_index_buy_sell:
    new_data_buy = list()
    for i, data in enumerate(data_index_buy_sell):
        if list(data_index_buy_sell[i].keys())[0] != "sold":
            new_data_buy.append(data)
    all_new_data_buy.append(new_data_buy)
#all_new_data_buy = all_data_index_buy_sell

all_df_data = list()
for c_i, candles in enumerate(all_candles):
    df_data = {"index": [], "result": [], "close": [],
               "open": [], "high": [], "low": [],
               "volume": []}
    counter = 0
    for i, candle in enumerate(list(candles)):
        df_data["index"].append(i)
        df_data["close"].append(float(quotation_to_decimal(candle.close)))
        df_data["open"].append(float(quotation_to_decimal(candle.open)))
        df_data["high"].append(float(quotation_to_decimal(candle.high)))
        df_data["low"].append(float(quotation_to_decimal(candle.low)))
        df_data["volume"].append(candle.volume)
        
        if list(all_new_data_buy[c_i][counter].keys())[0] == "bought" and i == all_new_data_buy[c_i][counter]["bought"]:
            df_data["result"].append(1)
            counter += 1 if counter < len(all_new_data_buy[c_i]) - 1 else 0
        # elif list(all_new_data_buy[c_i][counter].keys())[0] == "sold" and i == all_new_data_buy[c_i][counter]["sold"]:
        #     df_data["result"].append(1)
        #     counter += 1 if counter < len(all_new_data_buy[c_i]) - 1 else 0
        else:
            df_data["result"].append(0)
    all_df_data.append(df_data)
        
print(f"All different period data len: {len(all_df_data)}", f"true: {all_df_data[0]['result'].count(1)}", f"false: {all_df_data[0]['result'].count(0)}")

All different period data len: 10 true: 669 false: 2775


# Add Strategy Signals To DF

In [70]:
from Strategies.StrategyAroon import StrategyAroon
from Strategies.StrategyStochRSI import StrategyStochRSI
from Strategies.StrategyEMA import StrategyEMA

# Define strategy classes
strategy_classes = {
    "EMA": StrategyEMA,
    "RSI": StrategyRSI,
    "MACD": StrategyMACD,
    "BB": StrategyBB,
    "ST": StrategyST,
    "StochRSI": StrategyStochRSI,
    "Aroon": StrategyAroon,
    "OBV": StrategyOBV
}

# candle interval init (for this case is always 1 min)
candle_interval = CandleInterval.CANDLE_INTERVAL_1_MIN

# Initialize all strategies
strategies_blocks = []
for strategy_name, strategy_class in strategy_classes.items():
    strategies = [strategy_class() for _ in range(len(all_candles))]  # Create 7 instances of each strategy
    strategies_blocks.append(strategies)

# Initialize moving average containers
for strategy_block in strategies_blocks:
    for i, strategy in enumerate(strategy_block):
        put_in_candles = all_candles[i]
        strategy.initialize_moving_avg_container(put_in_candles[:strategy.history_candles_length])
    
print(f"Strategy blocks len: {len(strategies_blocks)}")    

Strategy blocks len: 8


In [71]:
all_df_data_st = [{} for i in range(len(strategies_blocks[0]))]

for strategy_block in strategies_blocks:
    for str_i, strategy_type in enumerate(strategy_block):
        for candle_index in range(strategy_type.history_candles_length, len(all_candles[str_i])):
            params = strategy_type.get_candle_param(all_candles[str_i][candle_index])
            for param_name in params:
                if candle_index == strategy_type.history_candles_length:
                    all_df_data_st[str_i][param_name] = list()
                    [all_df_data_st[str_i][param_name].append(None) for i in range(strategy_type.history_candles_length)]
                all_df_data_st[str_i][param_name].append(params[param_name])
            
print(f"Small length check should be the length of candles: {len(all_df_data_st[2]['BB_lower'])}, candle len: {len(all_candles[2])}")

Small length check should be the length of candles: 3280, candle len: 3280


In [72]:
# convert to df
for i in range(len(all_df_data)):
    all_df_data[i] = pd.DataFrame(all_df_data[i])

for i in range(len(all_df_data)):
    for name in all_df_data_st[i]:
        all_df_data[i][name] = all_df_data_st[i][name]

biggest_gap = 0
for strategy_block in strategies_blocks:
    for strategy_type in strategy_block:
        if strategy_type.history_candles_length > biggest_gap:
            biggest_gap = strategy_type.history_candles_length

# delete first NaN elements
for i, df in enumerate(all_df_data):
    all_df_data[i] = df.drop(df.index[:biggest_gap])

concatenated_df = pd.DataFrame()
for i, data in enumerate(all_df_data):
    data.to_csv(f"./marked_data/result_with_parameters_{i}.csv", index=False)
    
all_df_data[0]

,index,result,close,open,high,low,volume,EMA_prev_diff,EMA_current_diff,prev_RSI,...,BB_lower,ST_upper,ST_lower,ATR,StochRSI_K,StochRSI_D,aroon_up,aroon_low,OBV,OBV_MA
200,200,0,268.49,268.37,268.50,268.37,4941,1.232250,1.192782,46.354635,...,267.682235,272.205881,266.882279,1.326960,37.297297,21.986264,12.0,32.0,39873.0,3.696633e+04
201,201,0,268.52,268.50,268.72,268.43,11064,1.192782,1.153739,46.762067,...,267.674966,272.198489,266.882279,1.324496,38.918919,22.526804,8.0,28.0,50937.0,3.759467e+04
202,202,0,268.47,268.53,268.55,268.46,6896,1.153739,1.121937,46.830047,...,267.671207,272.151590,266.882279,1.308863,36.216216,21.625904,4.0,24.0,44041.0,3.927073e+04
203,203,0,268.45,268.50,268.51,268.43,12438,1.121937,1.094093,46.729598,...,267.762186,272.105817,266.882279,1.293606,35.135135,21.265543,4.0,20.0,31603.0,4.133033e+04
204,204,0,268.44,268.45,268.53,268.43,7620,1.094093,1.068967,46.689137,...,267.836687,272.067096,266.882279,1.280699,34.594595,21.085363,60.0,16.0,23983.0,4.161473e+04
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3439,3439,0,277.37,277.18,277.39,277.18,5145,-0.862271,-0.847275,51.690001,...,277.154747,271.975127,284.287148,1.182107,13.333333,13.998276,28.0,96.0,2251212.0,2.267610e+06
3440,3440,0,277.54,277.38,277.55,277.37,2374,-0.847275,-0.847539,52.239975,...,277.169895,271.975127,284.287148,1.181966,24.666667,17.776054,24.0,92.0,2253586.0,2.265855e+06
3441,3441,0,277.40,277.56,277.75,277.40,10999,-0.847539,-0.835031,52.702258,...,277.194850,271.975127,284.287148,1.193169,15.333333,14.664943,20.0,88.0,2242587.0,2.263139e+06
3442,3442,0,277.33,277.40,277.55,277.30,9562,-0.835031,-0.817066,52.281326,...,277.219059,271.975127,284.287148,1.196957,10.666667,13.109387,16.0,84.0,2233025.0,2.259981e+06
